<a href="https://www.kaggle.com/nobuaki1978/chatbot-dataset-break-data-analysis?scriptVersionId=87082958" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chatbot-dataset-break/Break-dataset/QDMR/test_lexicon_tokens.json
/kaggle/input/chatbot-dataset-break/Break-dataset/QDMR/dev.csv
/kaggle/input/chatbot-dataset-break/Break-dataset/QDMR/dev_lexicon_tokens.json
/kaggle/input/chatbot-dataset-break/Break-dataset/QDMR/train.csv
/kaggle/input/chatbot-dataset-break/Break-dataset/QDMR/test.csv
/kaggle/input/chatbot-dataset-break/Break-dataset/QDMR/train_lexicon_tokens.json
/kaggle/input/chatbot-dataset-break/Break-dataset/logical-forms/dev.csv
/kaggle/input/chatbot-dataset-break/Break-dataset/logical-forms/train.csv
/kaggle/input/chatbot-dataset-break/Break-dataset/logical-forms/test.csv
/kaggle/input/chatbot-dataset-break/Break-dataset/QDMR-high-level/test_lexicon_tokens.json
/kaggle/input/chatbot-dataset-break/Break-dataset/QDMR-high-level/dev.csv
/kaggle/input/chatbot-dataset-break/Break-dataset/QDMR-high-level/dev_lexicon_tokens.json
/kaggle/input/chatbot-dataset-break/Break-dataset/QDMR-high-level/train.csv
/kaggle/input/chat

In [3]:
pip install tflearn

     |████████████████████████████████| 107 kB 291 kB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=a091e6e72e11d9094b94e450274c86ca354e51c5b3c68f2a65eea606db74467b
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
Note: you may need to restart the kernel to use updated packages.


In [5]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow as tf
import random
import json
import pickle

In [28]:
# loading data from logical form
df_lf_dev = pd.read_csv('/kaggle/input/chatbot-dataset-break/Break-dataset/logical-forms/dev.csv')
#df_lf_dev.head()

In [29]:
df_lf_train = pd.read_csv('/kaggle/input/chatbot-dataset-break/Break-dataset/logical-forms/train.csv')
df_lf_test = pd.read_csv('/kaggle/input/chatbot-dataset-break/Break-dataset/logical-forms/test.csv')

In [11]:
print("size(total number of cells): ", df_lf_dev.size)
print("shape(rows x cols): ", df_lf_dev.shape)

size(total number of cells):  46314
shape(rows x cols):  (7719, 6)


In [30]:
df_lf_train.head()

,question_id,question_text,decomposition,program,operators,split
0,ACADEMIC_train_0,return me the homepage of PVLDB .,return homepages ;return #1 of PVLDB,"[""SELECT['homepages']"", ""FILTER['#1', 'of PVLD...","['select', 'filter']",train
1,ACADEMIC_train_1,"return me the homepage of "" H. V. Jagadish "" .",return homepages ;return #1 of H. V. Jagadish,"[""SELECT['homepages']"", ""FILTER['#1', 'of H. V...","['select', 'filter']",train
2,ACADEMIC_train_10,"return me the number of references of "" Making...",return references ;return #1 of Making databa...,"[""SELECT['references']"", ""FILTER['#1', 'of Mak...","['select', 'filter', 'aggregate']",train
3,ACADEMIC_train_100,return me the number of papers in VLDB confere...,return papers ;return #1 in VLDB conference ;r...,"[""SELECT['papers']"", ""FILTER['#1', 'in VLDB co...","['select', 'filter', 'filter', 'aggregate']",train
4,ACADEMIC_train_101,return me the number of authors who have paper...,return papers ;return #1 containing keyword Re...,"[""SELECT['papers']"", ""FILTER['#1', 'containing...","['select', 'filter', 'project', 'aggregate']",train


In [31]:
df_lf_test.head()

,question_id,question_text,decomposition,program,operators,split
0,ATIS_test_0,what flights go from dallas to phoenix,NaN,NaN,NaN,test
1,ATIS_test_1,what flights go from phoenix to salt lake city,NaN,NaN,NaN,test
2,ATIS_test_10,i would like flight information from phoenix t...,NaN,NaN,NaN,test
3,ATIS_test_100,which airline is kw,NaN,NaN,NaN,test
4,ATIS_test_101,please list all flights from new york to miami...,NaN,NaN,NaN,test


In [ ]:
# test.csv is an empty csv that can be used for testing the ML models created from dev and train.

In [32]:
print("train size(total number of cells): ", df_lf_train.size)
print("train shape(rows x cols): ", df_lf_train.shape)

train size(total number of cells):  264588
train shape(rows x cols):  (44098, 6)


In [35]:
pd.set_option('display.max_columns', None)  
pd.option_context('display.max_colwidth', None)
display(df_lf_train["program"][:10])

0    ["SELECT['homepages']", "FILTER['#1', 'of PVLD...
1    ["SELECT['homepages']", "FILTER['#1', 'of H. V...
2    ["SELECT['references']", "FILTER['#1', 'of Mak...
3    ["SELECT['papers']", "FILTER['#1', 'in VLDB co...
4    ["SELECT['papers']", "FILTER['#1', 'containing...
5    ["SELECT['papers']", "FILTER['#1', 'containing...
6    ["SELECT['organizations']", "AGGREGATE['count'...
7    ["SELECT['organizations']", "FILTER['#1', 'in ...
8    ["SELECT['Databases area']", "PROJECT['organiz...
9    ["SELECT['organizations']", "FILTER['#1', 'in ...
Name: program, dtype: object

In [36]:
display(df_lf_train["operators"][:10])

0                            ['select', 'filter']
1                            ['select', 'filter']
2               ['select', 'filter', 'aggregate']
3     ['select', 'filter', 'filter', 'aggregate']
4    ['select', 'filter', 'project', 'aggregate']
5    ['select', 'filter', 'project', 'aggregate']
6                         ['select', 'aggregate']
7               ['select', 'filter', 'aggregate']
8              ['select', 'project', 'aggregate']
9     ['select', 'filter', 'filter', 'aggregate']
Name: operators, dtype: object

In [38]:
import ast
df_lf_train["operators"].apply(ast.literal_eval)

0                                      [select, filter]
1                                      [select, filter]
2                           [select, filter, aggregate]
3                   [select, filter, filter, aggregate]
4                  [select, filter, project, aggregate]
                              ...                      
44093              [select, filter, project, aggregate]
44094              [select, filter, project, aggregate]
44095        [select, project, project, project, union]
44096         [select, filter, project, project, union]
44097    [select, project, aggregate, aggregate, union]
Name: operators, Length: 44098, dtype: object

In [39]:
df_lf_train['operators'].apply(lambda x: pd.Series(x.split(','))).stack().reset_index(level=1, drop=True)

0           ['select'
0           'filter']
1           ['select'
1           'filter']
2           ['select'
             ...     
44097       ['select'
44097       'project'
44097     'aggregate'
44097     'aggregate'
44097        'union']
Length: 209448, dtype: object

In [40]:
s = df_lf_train['operators'].apply(lambda x: pd.Series(x.split(','))).stack().reset_index(level=1, drop=True)
pd.crosstab(s.index, s)

col_0,'aggregate','aggregate'],'arithmetic','arithmetic'],'boolean','boolean'],'comparative','comparative'],'comparison'],'discard','discard'],'filter','filter'],'group','group'],'intersection','intersection'],'project','project'],'select','select'],'sort','sort'],'superlative','superlative'],'union','union'],['select',['select']
row_0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44093,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
44094,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
44095,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,1,1,0
